In [1]:
# dependencies

import os
os.environ['OPENBLAS_NUM_THREADS'] = '1'
os.environ['MKL_NUM_THREADS'] = '1'

import gzip
from pickle import dump, load

import numpy as np
import pandas as pd

from switch import mcmc
from switch.models import tanh

In [ ]:
# configure algorithm

n_samples = 100000
n_init = 10000
n_thin = 10
init_scale_h = 2
n_dc_splits = 4
pr_portkey_thi = 1e-3
seed = 0

# configure prior

prior_shape_jumps = 1
prior_rate_jumps = 48

In [3]:
# import data

f109 = pd.read_csv('paper/f109.csv', sep=',', parse_dates=['date']).iloc[:, 1:]
f109

,date,cumTime,centerE,centerN
0,2009-01-18,0.00000,0.000,0.000
1,2009-01-18,0.99984,0.018,-0.066
2,2009-01-18,1.99992,0.010,-0.078
3,2009-01-18,3.01656,0.030,-0.213
4,2009-01-23,123.73320,-0.381,-0.383
...,...,...,...,...
3914,2012-08-30,31672.71648,12.198,-10.904
3915,2012-08-30,31677.71664,12.033,-8.594
3916,2012-08-30,31683.08328,12.040,-8.597
3917,2012-08-30,31683.61656,12.041,-8.592


In [4]:
# select 2009 data

t = f109.cumTime.values[f109.date.dt.year == 2009]
vt = f109.centerE.values[f109.date.dt.year == 2009]

In [ ]:
# set up sampler

samples = []
sampler = mcmc.sample_posterior(
    t,
    vt,
    2,
    prior_shape_jumps,
    prior_rate_jumps,
    tanh.mod,
    np.random.default_rng(seed),
    init_scale_h=init_scale_h,
    n_init=n_init,
    n_dc_splits=n_dc_splits,
    pr_portkey_thi=pr_portkey_thi)

In [ ]:
# simulate markov chain

for _ in range(n_samples):
    s = next(sampler)
    if not _ % n_thin:
        samples.append(s[:3])

<lambdifygenerated-7>:3: RuntimeWarning: divide by zero encountered in log
  return b*r*(x + log(tanh(m - x) + 1))
/Users/tim.sf/Documents/research/ctbayes/markov_switch_remote/switch/factory.py:442: RuntimeWarning: invalid value encountered in subtract
  dsx_dil = self.eval_sx_dil(thi[yt[:-1]].T, xt[1:]) - self.eval_sx_dil(thi[yt[:-1]].T, xt[:-1])
/Users/tim.sf/Documents/research/ctbayes/markov_switch_remote/switch/misc/bfactories.py:59: RuntimeWarning: invalid value encountered in logaddexp
  log_prob = np.sum(log_odds) - np.logaddexp(0, np.sum(log_odds))
/Users/tim.sf/Documents/research/ctbayes/markov_switch_remote/switch/misc/bfactories.py:59: RuntimeWarning: invalid value encountered in scalar subtract
  log_prob = np.sum(log_odds) - np.logaddexp(0, np.sum(log_odds))


In [ ]:
# dump to disc

state = {k: v for k, v in sampler.gi_frame.f_locals.items() if k in
         ('thi', 'lam', 'h', 'z', 't', 'vt', 'hyper_lam', 'thi_samplers', 'node_sampler', 'ctrl', 'ome')}
with gzip.open('paper/f109_mcmc_state.pkl', 'wb') as f:
    dump(state, f)
with gzip.open('paper/f109_mcmc_samples.pkl', 'wb') as f:
    dump(samples, f)

In [ ]:
# to restart from saved state

# with gzip.open('paper/f109_mcmc_state.pkl', 'rb') as f:
#     state = load(f)
# with gzip.open('paper/f109_mcmc_samples.pkl', 'rb') as f:
#     samples = load(f)

# sampler = mcmc.resume_sampling(**state, mod=tanh.mod)